# Webbskrapning
Webbskrapning (eng. *web scraping*) är en teknik för att automatiskt ladda innehållet från webbsidor. 


# Grundläggande webbsidedesign

För att kunna hämta innehåll i webbsidor automatiskt, måste vi ha en förståelse för hur webbsidor är uppbyggda. Detta är inte en kurs i webbsidedesign, front-end, HTML eller Javascript, men det krävs en elementär förståelse av detta för att kunna hämta informationen.

På 90-talet var webbsidor nästan uteslutande uppbyggda av vad som kallas HTML (*hypertext markup language*), CSS, och ibland Javascript. Numer är webbsidor väsentligt mycket mer komplicerade, och kan ofta generera sitt innehåll dynamiskt beroende på användarens handlingar för maximal flexibilitet. Förr var en större del av webbsidor så kallat statiska. Statiska sidor är ungefär som pappersdokument i ett bibliotek, fast på internet - man skriver dem en gång och sedan visas de upp för besökare.

Även numer är många (de flesta) webbsidor statiska. Företag brukar emellertid använda sig av dynamiska webbsidor, som lever i ett gränsland mellan webbsidor och webbapplikationer, som exempelvis Facebook, Twitter och Reddit. Det är exempelvis ingen som fysiskt skrivit ner din profilsida på Facebook, utan denna är genererad dynamiskt från ditt angivna innehåll.


## HTML (HyperText Markup Language)
HTML är ett såkallat markup-språk, en samling enkla kommandon för att formatera text på hemsidor. Det utgår från vad som kallas *taggar* eller *element*, en inledande och avslutande tagg som omsluts av pilparenteser (``<span>...</span>, <html>...</html>``) och innehåller någon form av text eller annat innehåll. Taggarna används för att specificera vilket typ av innehåll som skall rymmas inuti, och vilken stil det ska.

Exempel:
- ``<html>``: Betecknar huvudblocket av HTML, och säger åt datorn att det rör sig om HTML.
- ``<body>``: Brukar användas för brödtext/-innehållet på en sida. Liksom i typografi finns också ``<header>`` och ``<footer>``.
- ``<div>``: "Division", eller avdelare, används för att dela upp en webbsida i block.
- ``<b>``, ``<i>``, ``<u>``: Formaterare av text för fetstil, snedstil respektive understruken stil.
- ``<a>``: Betecknar en hyperlänk, eller URL.
- ``<span>``: Används för att formatera ett stycke i en text
- ``<article>``: En tagg för att beteckna "artiklar", exempelvis blogginlägg, självständiga samlingar av teststycken.

Av ren händelse är Jupyter Notebooks skrivna i HTML, och kan formatera det utan problem:

In [10]:
%%html
<html>
    <body>

        <div>Hello World!</div>

        <div><b>Hello</b> World!</div>

        <div><i>Hello</i> World!</div>

        <div><a href="https://en.wiktionary.org/wiki/hello">Hello</a> World!</div>

    </body>
</html>

Vissa taggar, som ``<article>`` eller ``<div>`` gör ingenting i isolation. Det är konvention att man fyller en ``<div>`` med andra HTML-komponenter (så som kolumner) och ``<article>`` med just artikelinnehåll. Dessa kan formateras med ett separat dokument, exempelvis skriver i CSS.

### Klasser och ID:n
Vi ska inte diskutera CSS och stilar i detta kursmoment, det finns folk som gör det väsentligt mycket bättre än jag. Däremot skall vi diskutera användandet av så kallade *klasser* och *ID* i HTML. Dessa tillåter egenskapade taggar som kan styras som grupp (s.k. klasser) eller på individuell nivå (med hjälp av ID).

Vi modifierar våra minimala exempel ovan genom att lägga till en ``<style>``-tagg, som formaterar klassen ``.tomat`` och elementet ``#sista`` med CSS. ``<style>``-taggen läggs i princip alltid i en ``<head>``-tagg som läggs högst upp i dokumentet. Resultatet blir att varje avdelare av klass ``tomat`` modifieras till stilen preciserad i formatteringen. Därtill skrivs stilen på sista elementet över med den individuella, ljusgröna bakgrundsformatteringen.

In [1]:
%%html

<html>
  <head>
    <style>
      .tomat {
        background-color: tomato;
        color: white;
        border: 2px solid black;
        margin: 5px;
        padding: 5px;
      }

      #sista {
        background-color: lightgreen;
      }
    </style>
  </head>

  <body>
    <div class='tomat'>
      Hello World!
    </div>

    <div class='tomat'>
      <b>Hello</b> World!
    </div>

    <div class='tomat'>
      <i>Hello</i> World!
    </div>

    <div class='tomat' id='sista'>
      <a href="https://en.wiktionary.org/wiki/hello">Hello</a> World!
    </div>

  </body>
</html>

# Demo: Webbsida

Ni kan inspektera hur en webbsida är uppbyggd genom att navigera in på valfri webbsida och klicka ``Ctrl+Shift+I``, alternativt högerklicka och välja "Inspektera/Inspect" i menyn.


<a href=https://www.europeana.eu/en/blog/love-for-life-the-mandarin-duck>Love for life: the mandarin duck</a>

## Webbskrapning
Webbskrapning är en bred term för att ladda ner information från webbsidor. Det kan röra sig om att ladda ner hela sidan så troget som möjligt (vilket ofta omfattas av webbarkiv-projekt), till att endast ladda ner specifika element/taggar.

Skrapning kan göras manuellt: Genom att kopiera innehållet som tidigare demonstrerat. Det är oftast inte vad som avses dock, utan snarare avses automatisk skrapning av många sidor på en *domän*. En domän är den adress alla sidor utgår ifrån (exempelvis ``wikipedia.org``, ``en.wikipedia.org``, ``dh.gu.se``. ``gu.se``, ``twitter.com``). Oftast brukar man vara intresserad av att hålla sig inom en domän, men man kan exempelvis tänka sig ett mål att skapa ett nätverk av hur webbsidor referar till varandra.

Automatiserad webbskrapning fungerar på exakt samma sätt som manuell sådan - man måste först inspektera sin sida, och hitta de element man är intresserad av.

Liksom i verkliga livet  vi två komponenter: En webbläsare, som öppnar själva sidan, samt någon som letar efter de element vi är intresserade av.

- ``requests`` är en webbläsare i kodform - den öppnar sidor och håller den i minnet.
- ``BeautifulSoup`` eller ``bs4`` är den funktion som indexerar innehållet och låter oss automatiskt hitta en viss komponent i sidan vi redan vet finns där. Den skapar vad som kallas "html-soppor", vilket är det skämtsamma namnet på den rå, oläsbara html:n.

In [10]:
import requests
import bs4

# Ange webaddressen till sidan
page = 'https://www.europeana.eu/en/blog/love-for-life-the-mandarin-duck'

# Läs in sidan
response = requests.get(page)

# Hämta soppan av HTML
soup = bs4.BeautifulSoup(response.text, features="lxml")

In [12]:
print(soup.prettify())

<!DOCTYPE html>
<html data-n-head="%7B%22lang%22:%7B%22ssr%22:%22en-GB%22%7D%7D" data-n-head-ssr="" lang="en-GB">
 <head>
  <title>
   Love for life: the mandarin duck | Europeana
  </title>
  <meta charset="utf-8" data-n-head="ssr"/>
  <meta content="width=device-width, initial-scale=1" data-n-head="ssr" name="viewport"/>
  <meta content="https://www.europeana.eu/en/blog/love-for-life-the-mandarin-duck" data-hid="og:url" data-n-head="ssr" property="og:url"/>
  <meta content="en_GB" data-hid="og:locale" data-n-head="ssr" property="og:locale"/>
  <meta content="bg_BG" data-hid="og:locale:alternate-bg-BG" data-n-head="ssr" property="og:locale:alternate"/>
  <meta content="cs_CZ" data-hid="og:locale:alternate-cs-CZ" data-n-head="ssr" property="og:locale:alternate"/>
  <meta content="da_DK" data-hid="og:locale:alternate-da-DK" data-n-head="ssr" property="og:locale:alternate"/>
  <meta content="de_DE" data-hid="og:locale:alternate-de-DE" data-n-head="ssr" property="og:locale:alternate"/>
  

In [22]:
soup.find_all('p')

[<p class="lead">
             A bright-coloured mohawk, green forehead, orange sideburns, purple collar and perching toes: not your idea of a dream date? The mandarin duck could make you reconsider!
           </p>,
 <p>Apart from being considered among the most beautiful bird species in the world, this native of China - sometimes called ‘The Far East Rainbow’ or ‘the Yin-Yang duck’ - is also believed to be the most faithful. </p>,
 <p>For centuries, mandarin duck pairs were thought to stay together for life. Yet in reality the <em><a href="https://en.wikipedia.org/wiki/Mandarin_duck">Aix galericulata</a></em> is more capricious than that, pairing for the season yet moving on to a new partner in autumn.</p>,
 <p>Still, in Asian culture, the mandarin ducks remain the prime symbol of love, fidelity and fertility.</p>,
 <p>When it comes to making a dashing impression, the drake mandarin outshines his female companion. </p>,
 <p>He’s the one making all the noise - females don’t tend to qu

In [20]:
print(soup.get_text())



Love for life: the mandarin duck | Europeana


  
    Skip to page contents
        
        Home
       
        Collections
        
          Login/Join
             
        Home
       
        Collections
       
        Teachers
       
        About us
       
        Help
        
          Login/Join
          BlogLove for life: the mandarin duck  
            Love for life: the mandarin duck
           
            A bright-coloured mohawk, green forehead, orange sideburns, purple collar and perching toes: not your idea of a dream date? The mandarin duck could make you reconsider!
             
              Published February 12, 2021
             
              by
             
    Sofie Taes
   (KU Leuven / Photoconsortium) 
  Share
  Apart from being considered among the most beautiful bird species in the world, this native of China - sometimes called ‘The Far East Rainbow’ or ‘the Yin-Yang duck’ - is also believed to be the most faithful. 
For centuries, mandarin duck

In [24]:
soup.find_all('a', href=True)

[<a class="skip-main" data-qa="main content accessibility link" href="#main">
     Skip to page contents
   </a>,
 <a class="logo d-inline-flex nuxt-link-active" data-v-2190d19a="" href="/en" target="_self"><img alt="Europeana home" class="mw-100" data-qa="logo" data-v-2190d19a="" src="/_nuxt/img/logo.4c70f4c.svg"/></a>,
 <a class="nuxt-link-active nav-link" data-v-996eff90="" exact="" href="/en/" target="_self"><span class="nav-link-icon icon-home" data-v-996eff90=""></span> <span data-v-996eff90="">
         Home
       </span></a>,
 <a class="nav-link" data-v-996eff90="" exact="" href="/en/collections" target="_self"><span class="nav-link-icon icon-collections" data-v-996eff90=""></span> <span data-v-996eff90="">
         Collections
       </span></a>,
 <a class="nav-link" data-qa="log in button" data-v-996eff90="" href="/account/login" target="_self"><span class="nav-link-icon icon-login" data-v-996eff90=""></span> <span data-v-996eff90="">
           Login/Join
         </span></

In [25]:
[ a['href'] for a in soup.find_all('a', href=True) ]

['#main',
 '/en',
 '/en/',
 '/en/collections',
 '/account/login',
 '/en',
 '/en/',
 '/en/collections',
 '/en/europeana-classroom',
 '/en/about-us',
 '/en/help',
 '/account/login',
 '/blog',
 'https://pro.europeana.eu/person/sofie-taes',
 'https://en.wikipedia.org/wiki/Mandarin_duck',
 'http://datazone.birdlife.org/species/factsheet/22680107',
 'https://en.wikipedia.org/wiki/Central_Park_mandarin_duck',
 'https://www.europeana.eu/chinese-heritage',
 'https://pro.europeana.eu/project/pagode-europeana-china']

## Webbskrapning och etik
Med nästan alla datahantering kommer lager av etik och legalitet. Viss data har mycket liten etisk påverkan, annan har väldigt stor. Data som rör människor och personuppgifter anses synnerligen känslig, och är reglerad i lag, i Sverige av GDPR och personuppgiftslagen PUL. I exempelvis USA finns andra (i det fallet ofta mer följsamma) lagar. 

Grovt uppdelat bör man som forskare reflektera över tre stadier av etisk databehandling. 

- Insamlingsetik: Hur samlas data in med hänsyn till integritet och lagar? Exempelvis personuppgifter i en enkätundersökning.
- Behandlingsetik: Vad gör man med datan? Hur modifieras den, i vilket projekt används den? Exempelvis skillnaden på om en polisforskare samlar in etnisk information och om en etnolog gör det.
- Konsekvensetik: Vad är resultatet av undersökningen? Hur påverkar det samhällsklimatet, vilka konsekvenser för i synnerhet människor har resulteten?

Detta är en föreläsning i sig, och vi skall fokusera på den första delen: Insamlingsetik.

### Att samla in data med skrapning
När data samlas in med hjälp av skrapning måste man vara medveten vad som faktiskt samlas in - något som oftast är per definition omöjligt i en automatiserad process. Det är inte möjligt att ha insyn i exakt vad som samlas in, vilket kan inkludera privata detaljer, lösenord eller personuppgifter. 

Att endast ladda ner sådant material är inte olagligt, i synnerhet inte ifall det är allmänt publicerat (avsiktligen eller oavsiktligen). Att återpublicera och tillgängliggöra sånt material (exempelvis som ett dataset) är däremot olagligt enligt bl. a. GDPR. 

Den huvudskliga frågan är om det är etiskt att utnyttja att sådan information är allmänt tillgänglig. Om e-postdata blev allmänt tillgänglig, är det då lämpligt att skrapa denna och använda i sin forskning?

### Copyright
Copyright är en samling lagar som styr rätten att reproducera bl. a. skriven text. Den mesta nätpublicerade texten ligger under så kallad *fair use*, vilket bland annat innebär att den får utnyttjas för forskning (men kanske inte av myndigheter eller industri). Om ni har frågor gällande detta för ett eventuellt större projekt bör ni kontakta Universitetsbiblioteket eller GU:s jurister. Det är viktigt att inse att copyright-lagar skiljer sig mellan länder, men att du som individ i regel endast kan lagföras i ditt eget land.


### Belastning
Att ladda ner och traversera alla sidor på en domän tillför ansenlig belastning på de datorer och servrar som *hostar* webbsidorna. Webbsidor har oftast ett fast antal besökare som kan stödjas i en tidsperiod, och om man överskrider denna gräns kan sidan bli långsammare att ladda och som värst behöva stänga ner. Det kan också tillföra en kostnad till sidans ägare. När detta görs avsiktligen kallas det vanligen för en *DoS*- (*denial of service*) -attack. Samma effekt kan dock uppnås av storskalig skrapning.

Även om denna kostnad är mycket liten för ett företag som exempelvis Twitter och Facebook, är deras tjänster beroende av att tillhandahålla en viss hastighet, och dessa företag liksom många andra kommer att strypa hastigheten hos skrapningsverktyg, och till slut blockera dem.

Den tillförda belastningen har också en mer esoterisk konsekvens som börjat bli aktuell endast de senaste åren: Ju större och mer avancerade webbsidorna blir, desto större beräkningskraft krävs för att hitta och ladda ner material. Detta genererar en numer nollskild elräkning, och därmed klimatskuld för upprepade, storskaliga skrapningar.

## Webb-API
Ett webb-API, eller *application programming interface* är ett programmeringsgränsnitt (en kommunikation mellan programmeringspråk- och program) som möjliggör att program eller databaser på olika datorer kan dela data via webb-anrop. Det betyder i princip att man kan hämta data direkt genom att skriva in en webb-adress (URI) i webbläsaren. Detta är ett säkert och snabbt sätt att ladda ner data från hemsidor som tillåter det. På detta vis kan källan till datan begränsa vad som är tillgängligt och det behövs väsentligt mindre programmering och resurser för att hämta datan än via skrapning. **API:n är alltså ett hållbart alternativ till skrapning.**

### Twitter API
Twitter har ett eget API för att ladda ner data, eftersom de annars inte tillåter att man skrapar på domänen. Problemet med detta API är att man 1) behöver ett konto, 2) det är kraftigt begränsat, och man kan endast skrapa ett begränsat antal tweets i timmen. Å andra sidan kommer du aldrig att bli blockad.

### Wordpress API
Vi tar ett tydligt exempel i form av Wordpress. Wordpress är den absolut största plattformen för hemsidor, och framför allt bloggande. 

Tills vidare begränsar vi oss till bloggar och den information man möjligtvis skulle kunna utvinna därifrån - bloggar innehåller text, inläggsförfattare, datum och olika ämnen. Detta möjliggör utforskning av texten över tid, ämnesutforskning med exempelvis *topic models* eller kanske association av författarna med särskilt språk eller ämne. En annan möjlighet är att bygga nätverk av författarna och de personer som nämns i deras texter.

Möjligheterna är oändliga, som det heter. Det är också tyvärr så att Wordpress-bloggar har väldigt unik konstruktion, och man kan behöva inspektera HTML-strukturen och anpassa sina skrapningsalgoritmer efter varje enskild blogg. Wordpress begränsar också tillgången för de datorer som missbrukar sidorna, exempelvis högintensiv skrapning. Wordpress tillhandahåller istället publika API:n genom vilka man kan få tillgång till blogginläggen i klartext.

``http://www.feministmormonhousewives.org/wp-json/wp/v2/posts?search=god&per_page=5&categories=1``

Det består av två delar:


1. API *endpoint* avslutas med frågetecken: ``http://www.feministmormonhousewives.org/wp-json/wp/v2/posts`` (specifikt API för inlägg/posts)
2. Parametrar, varje parameter separeras med ett ampersand ``&``. I detta fall ``search``, ``per_page``, ``categories`` med värdena sökningen ``god``, antal träffar per sida ``5`` och kategori med nummer ``1``.


Det är viktigt att inse att alla organisationer och hemsidor som tillhandahåller ett API fungerar på olika sätt. Det är olika API endpoints och olika parametrar.

### Kungliga biblioteket LIBRIS API
KB LIBRIS har ett utmärkt webb-API för att hämta information om objekten i deras samlingar. Vi tar detta som exempel. Följande adress returnerar en sökning bland författare efter Ludwig Wittgenstein. Dessa tillåter att man avgränsar sökningen efter författare genom en speciell formattering på sökningen.

``http://libris.kb.se/xsearch?query=förf:(Ludwig%20Wittgenstein)&format=dc``

1. API: ``http://libris.kb.se/xsearch?``
2. Parametrar:  I detta fall en sökning enligt deras sökspråk ``query=förf:(Ludwig%20Wittgenstein)`` samt formatet på resultatet ``format=dc``


